In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

user = 'root'
password = 'sqlpassword:=9SfuA7'
host = 'localhost'
database = 'sakila'

engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

def rentals_month(engine, month, year):
    query = text("""
        SELECT 
            rental_id,
            customer_id,
            rental_date,
            return_date
        FROM rental
        WHERE MONTH(rental_date) = :month
          AND YEAR(rental_date) = :year
    """)
    
    df = pd.read_sql_query(query, engine, params={"month": month, "year": year})
    return df


def rental_count_month(df, month, year):
    column_name = f"rentals_{month:02d}_{year}"
    
    rental_count = (
        df.groupby("customer_id")["rental_id"]
          .count()
          .reset_index()
          .rename(columns={"rental_id": column_name})
    )
    
    return rental_count


def compare_rentals(df1, df2):
    combined = pd.merge(df1, df2, on="customer_id", how="inner")
    
    cols = [c for c in combined.columns if c.startswith("rentals_")]
    combined["difference"] = combined[cols[1]] - combined[cols[0]]
    
    return combined
